Jai shree ram

In [1]:
import os
import pandas as pd
import csv
import numpy as np

def mapLabels(label:str):
    if label == "boxing":
        return 0
    elif label == "drums":
        return 1
    elif label == "guitar":
        return 2
    elif label == "rowing":
        return 3
    elif label == "violin":
        return 4
    
def removeConfidenceScore(keypoints):
    # Remove every kth item in-place
    K = 3
    del keypoints[K-1::3] 

    return keypoints

    

def removeKeypoints(keypoints):
    removeIdx = [0,1,4,7,8,9,11,12,14,15,16,17,18,19,20,22,23]
    removeIdx.reverse()

    kp = np.array(keypoints).reshape(-1,3)

    for idx in removeIdx:
        kp = np.delete(kp,idx,0)
    
    kp = kp.flatten()

    return kp.tolist()

    
def normalizeKeyPoints(keyPoints):
    poseKeyPoints = np.array(keyPoints).reshape(-1,3)

    # Here we extracr x,y coordinates
    x_coordinates = poseKeyPoints[:, 0]
    y_coordinates = poseKeyPoints[:, 1]

    x_coordinates_normalized = (x_coordinates - np.min(x_coordinates)) / (np.max(x_coordinates) - np.min(x_coordinates))
    y_coordinates_normalized = (y_coordinates - np.min(y_coordinates)) / (np.max(y_coordinates) - np.min(y_coordinates))

    # Replace the normalized x, y coordinates in the original array
    poseKeyPoints[:, 0] = x_coordinates_normalized
    poseKeyPoints[:, 1] = y_coordinates_normalized

    poseKeyPoints = poseKeyPoints.flatten()

    return poseKeyPoints.tolist()



def importData(dirPath:str):
    """
    Returns the dataset as object from the 'dirPath'
    """

    trainSet = []

    trainPath = dirPath + '/train/train/'

    for file in os.listdir(trainPath):
        fileName = os.fsdecode(file)

        # Extract label from file name
        label =  fileName.split("_")[1].split(".")[0]
        # Here we map labeel to int 
        y = mapLabels(label)

        # Create a file object to parse csv files
        file = open(trainPath + fileName,newline='',encoding='utf-8')
        reader = csv.reader(file)
        samples = []
        
        # Extract 'keypoints, 'confidence' and 'angles'
        for row in reader:
            if len(row) == 79:
                keypoints = [float(row[i]) for i in range(0,75)]
                angles = [float(row[j]) for j in range(75,79)]

                keypoints = removeKeypoints(keypoints)

                poseKeypoints = normalizeKeyPoints(keypoints)

                #Remove confidence score
                poseKeypoints =  removeConfidenceScore(poseKeypoints)

                samples.append(poseKeypoints + angles)
        
        trainSet.append((samples,y))
    
    return trainSet

def importTestData(dirPath:str):
    """
    """

    testSet = []

    testPath = dirPath + '/test/test/'

    # Sort files
    fileList = sorted(os.listdir(testPath),key=lambda x:int(x.split('.')[0]))

    for file in fileList:
        fileName = os.fsdecode(file)

        # Extract label from file name
        id =  fileName.split(".")[0]

        # Create a file object to parse csv files
        file = open(testPath + fileName,newline='',encoding='utf-8')
        reader = csv.reader(file)
        samples = []
        
        # Extract 'keypoints, confidence' and 'angles'
        for row in reader:
            if len(row) == 79:
                keypoints = [float(row[i]) for i in range(0,75)]
                angles = [float(row[j]) for j in range(75,79)]
                keypoints = removeKeypoints(keypoints)
                poseKeypoints = normalizeKeyPoints(keypoints)

                #Remove confidence score
                poseKeypoints =  removeConfidenceScore(poseKeypoints)

                samples.append(poseKeypoints + angles)

        
        testSet.append((samples,id))
    
    return testSet

def getDataDistribution(data):
    distribution = []

    for d in data:
        sample = {'label':d[1],'count':len(d[0])}
        distribution.append(sample)

    return distribution

In [22]:
def create_sequences(data, time_steps=1):
    sequences = []
    labels = []
    for sample, label in data:
        for i in range(len(sample) - time_steps + 1):
            sequence = sample[i:(i + time_steps)]
            sequences.append(sequence)
            labels.append(label)
    return np.array(sequences), np.array(labels)

time_steps = 10  
trainSet = importData("/kaggle/input/lsd-portfolio-3")
X_train, y_train = create_sequences(trainSet, time_steps)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout


feature_dim = X_train.shape[2]

model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True), input_shape=(time_steps, feature_dim)),
    Dropout(0.2),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.2),
    Bidirectional(LSTM(128)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')  # Assuming 5 classes based on your label mapping
])

model.summary()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_7 (Bidirecti  (None, 10, 256)           152576    
 onal)                                                           
                                                                 
 dropout_5 (Dropout)         (None, 10, 256)           0         
                                                                 
 bidirectional_8 (Bidirecti  (None, 10, 256)           394240    
 onal)                                                           
                                                                 
 dropout_6 (Dropout)         (None, 10, 256)           0         
                                                                 
 bidirectional_9 (Bidirecti  (None, 256)               394240    
 onal)                                                           
                                                      

In [41]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_7 (Bidirecti  (None, 10, 256)           152576    
 onal)                                                           
                                                                 
 dropout_5 (Dropout)         (None, 10, 256)           0         
                                                                 
 bidirectional_8 (Bidirecti  (None, 10, 256)           394240    
 onal)                                                           
                                                                 
 dropout_6 (Dropout)         (None, 10, 256)           0         
                                                                 
 bidirectional_9 (Bidirecti  (None, 256)               394240    
 onal)                                                           
                                                      

In [42]:
def create_sequences_test(data, time_steps=1):
    sequences = []
    ids = []
    for sample, id in data:
        num_sequences = len(sample) - time_steps + 1
        for i in range(num_sequences):
            sequence = sample[i:(i + time_steps)]
            sequences.append(sequence)
            ids.append(id)  
    return np.array(sequences), ids

testSet = importTestData("/kaggle/input/lsd-portfolio-3")
X_test, test_ids = create_sequences_test(testSet, time_steps)

In [43]:
# Make predictions
predictions = model.predict(X_test)

predicted_labels = np.argmax(predictions, axis=1)


3154/3154 [==============================] - 56s 18ms/step


In [44]:

import csv

predicted_labels = np.argmax(predictions, axis=1)

submission_data = zip(test_ids, predicted_labels)

# Create a submission file
submission_file = "submission11.csv"
with open(submission_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["id", "action"])
    for id, label in submission_data:
        writer.writerow([id, label])

print(f"Submission file created: {submission_file}")



Submission file created: submission11.csv


This is to run on the submission file generated....applies to everything

In [45]:
import csv
from collections import Counter

# Load the predictions from the CSV file
file_path = '/kaggle/working/submission6.csv'

predictions_by_id = {}

with open(file_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        id, prediction = int(row[0]), int(row[1])
        if id not in predictions_by_id:
            predictions_by_id[id] = []
        predictions_by_id[id].append(prediction)

# Find the most common prediction for each ID
final_predictions = {id: Counter(preds).most_common(1)[0][0] for id, preds in predictions_by_id.items()}

# Create a new submission file with these most common predictions
submission_file = 'submission_final.csv'
with open(submission_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["id", "action"])
    for id in range(305):  # Assuming IDs from 0 to 304
        writer.writerow([id, final_predictions.get(id, 0)])  # Default to 0 if no prediction

print(f"Final submission file created: {submission_file}")


Final submission file created: submission_final.csv
